In [1]:
import cv2
import numpy as np

APPLYING WITH L1 and L2

In [2]:
def distance_l1(x, y):
    return abs(x - y)

def distance_l2(x, y):
    return (x - y) ** 2

In [14]:
def window_based_matching(left_image, right_image, disparity_range, save_result=True, kernel_size=5, method="l1"):
    # Read left, right image then convert to grayscale
    left_image = cv2.imread(left_image, cv2.IMREAD_GRAYSCALE).astype(np.float32)
    right_image = cv2.imread(right_image, cv2.IMREAD_GRAYSCALE).astype(np.float32)

    # Get the shape of the images
    height, width = left_image.shape[:2]

    # Create a blank image to store the result
    depth_map = np.zeros((height, width), dtype=np.uint8)
    kernel_half = int((kernel_size - 1) / 2)
    scale = 3
    max_value = 255 * 9

    for y in range(kernel_half, height - kernel_half + 1):
        for x in range(kernel_half, width - kernel_half + 1):
            # Find j where cost has minimum value
            disparity = 0
            cost_min = 65534
            for j in range(disparity_range):
                total = 0
                value = 0
                for v in range(-kernel_half, kernel_half):
                    for u in range(-kernel_half, kernel_half):
                        value = max_value
                        if (x + u - j) >= 0:
                            if method == "l1":
                                value = distance_l1(int(left_image[y + v, x + u]), int(right_image[y + v, (x + u) - j]))
                            elif method == "l2":
                                value = distance_l2(int(left_image[y + v, x + u]), int(right_image[y + v, (x + u) - j]))
                            total += value

                if total < cost_min:
                    cost_min = total
                    disparity = j

            # Let depth at (y, x) = j (disparity)
            # Multiply by scale factor for visualization purpose
            depth_map[y, x] = scale * disparity

    if save_result:
        print("Saving depth map...")
        if method == "l1":
            cv2.imwrite(f"window_based_l1.png", depth_map)
            cv2.imwrite(f"window_based_l1_color.png", cv2.applyColorMap(depth_map, cv2.COLORMAP_JET))
        elif method == "l2":
            cv2.imwrite(f"window_based_l2.png", depth_map)
            cv2.imwrite(f"window_based_l2_color.png", cv2.applyColorMap(depth_map, cv2.COLORMAP_JET))
    return depth_map

In [15]:
left_image = "./Aloe_images/Aloe/Aloe_left_1.png"
right_image = "./Aloe_images/Aloe/Aloe_right_1.png"
disparity_range = 64
kernel_size = 3

window_based_matching(left_image, right_image, disparity_range, save_result=True, kernel_size=3, method="l1")
window_based_matching(left_image, right_image, disparity_range, save_result=True, kernel_size=3, method="l2")

Saving depth map...
Saving depth map...


array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   6,   9, ...,  45,  48,  48],
       [  0,   6,   9, ...,  45,  48,  51],
       ...,
       [  0,   6,   9, ..., 102, 105,  72],
       [  0,   6,   9, ...,  66,  69, 129],
       [  0,   6,   9, ..., 129, 162, 129]], shape=(370, 427), dtype=uint8)

In [17]:
left_image = "./Aloe_images/Aloe/Aloe_left_1.png"
right_image = "./Aloe_images/Aloe/Aloe_right_1.png"
disparity_range = 64
kernel_size = 5

window_based_matching(left_image, right_image, disparity_range, save_result=True, kernel_size=3, method="l1")
window_based_matching(left_image, right_image, disparity_range, save_result=True, kernel_size=3, method="l2")

Saving depth map...
Saving depth map...


array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   6,   9, ...,  45,  48,  48],
       [  0,   6,   9, ...,  45,  48,  51],
       ...,
       [  0,   6,   9, ..., 102, 105,  72],
       [  0,   6,   9, ...,  66,  69, 129],
       [  0,   6,   9, ..., 129, 162, 129]], shape=(370, 427), dtype=uint8)

APPLYING COSINE SIMILARITY

In [29]:
def cosine_similarity(x, y):
    numerator = np.dot(x, y)
    denominator = np.linalg.norm(x) * np.linalg.norm(y)
    return numerator / denominator

In [33]:
def window_based_matching_cs(left_image, right_image, disparity_range, save_result=True, kernel_size=5):
    # Read left, right image then convert to grayscale
    left_image = cv2.imread(left_image, cv2.IMREAD_GRAYSCALE).astype(np.float32)
    right_image = cv2.imread(right_image, cv2.IMREAD_GRAYSCALE).astype(np.float32)

    # Get the shape of the images
    height, width = left_image.shape[:2]

    # Create a blank image to store the result
    depth_map = np.zeros((height, width), dtype=np.uint8)
    kernel_half = int((kernel_size - 1) / 2)
    scale = 3
    max_value = 255 * 9

    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            # Find j where cost has minimum value
            disparity = 0
            cost_optimal = -1
            for j in range(disparity_range):
                d = x - j
                cost = -1
                if (d - kernel_half) > 0:
                    wp = left_image[(y - kernel_half):(y + kernel_half + 1), (x - kernel_half):(x + kernel_half + 1)]
                    wqd = right_image[(y - kernel_half):(y + kernel_half + 1), (d - kernel_half):(d + kernel_half + 1)]
                    wp_flatten = wp.flatten()
                    wqd_flatten = wqd.flatten()
                    cost = cosine_similarity(wp_flatten, wqd_flatten)

                if cost > cost_optimal:
                    cost_optimal = cost
                    disparity = j

            depth_map[y, x] = scale * disparity
    
    if save_result:
        print("Saving depth map...")
        cv2.imwrite(f"window_based_cs.png", depth_map)
        cv2.imwrite(f"window_based_cs_color.png", cv2.applyColorMap(depth_map, cv2.COLORMAP_JET))
    return depth_map

In [34]:
left_image = "./Aloe_images/Aloe/Aloe_left_1.png"
right_image = "./Aloe_images/Aloe/Aloe_right_1.png"
disparity_range = 64
kernel_size = 3

window_based_matching_cs(left_image, right_image, disparity_range, save_result=True, kernel_size=3)

Saving depth map...


array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,  45,  45,   0],
       [  0,   0,   0, ...,  45,  48,   0],
       ...,
       [  0,   0,   0, ..., 126, 126,   0],
       [  0,   0,   0, ..., 129, 126,   0],
       [  0,   0,   0, ...,   0,   0,   0]], shape=(370, 427), dtype=uint8)